In [7]:
%matplotlib inline
import numpy as np
import pandas as pd
import datetime  
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix 

#### 1、数据预处理

In [8]:
path_train_1 = "./data/Train15p.csv"  # 训练文件1
path_train_2 = "./data/Train85p.csv"  # 训练文件2
path_test = "./data/Test.csv"         # 测试文件

In [9]:
df_train_1 = pd.read_csv(path_train_1)

In [10]:
df_train_1.Churn.value_counts()

 False    515
 True      85
Name: Churn, dtype: int64

In [16]:
def prod_trian_data_1(path):
    """
    处理训练数据集1
    :return: 
    """
    df = pd.read_csv(path)
    df =df.loc[:,[ 'account_length', 'international_plan', 'voice_mail_plan', 'number_vmail_messages',
           'total_day_minutes', 'total_day_calls', 'total_day_charge',
           'total_eve_minutes', 'total_eve_calls', 'total_eve_charge',
           'total_night_minutes', 'total_night_calls', 'total_night_charge',
           'total_intl_minutes', 'total_intl_calls', 'total_intl_charge','Churn']]
    df['international_plan'] = df['international_plan'].map(lambda x:1 if x==' yes' else 0)
    df['voice_mail_plan'] = df['voice_mail_plan'].map(lambda x:1 if x==' yes' else 0)
    df['Churn'] = df['Churn'].map(lambda x:1 if x==' True' else 0)
    
    data_train = df
    
    from sklearn.model_selection import train_test_split
    X = data_train.iloc[:, :-1]
    y = data_train.iloc[:,-1]

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4, random_state=8)

#     df_train = pd.concat([y_train, X_train], axis=1)
#     df_val = pd.concat([y_val, X_val], axis=1)
    return X_train, X_val, y_train, y_val,data_train

In [17]:
def prod_trian_data_2(path):
    """
    处理训练数据集2
    """
    df = pd.read_csv(path)
    df =df.loc[:,[ 'account_length', 'international_plan', 'voice_mail_plan', 'number_vmail_messages',
           'total_day_minutes', 'total_day_calls', 'total_day_charge',
           'total_eve_minutes', 'total_eve_calls', 'total_eve_charge',
           'total_night_minutes', 'total_night_calls', 'total_night_charge',
           'total_intl_minutes', 'total_intl_calls', 'total_intl_charge']]
    df['international_plan'] = df['international_plan'].map(lambda x:1 if x==' yes' else 0)
    df['voice_mail_plan'] = df['voice_mail_plan'].map(lambda x:1 if x==' yes' else 0)
     
    X_test =df
    
    return df

In [18]:
X_train, X_val, y_train, y_val,data_train = prod_trian_data_1(path_train_1)

In [27]:
y_train.value_counts()

0    314
1     46
Name: Churn, dtype: int64

In [19]:
X_train_2= prod_trian_data_2(path_train_2)

#### 2、建模

In [22]:
# 模型建立 xgboost
from xgboost import XGBClassifier
xgbr = XGBClassifier()
xgbr.fit(X_train, y_train)
y_pred = xgbr.predict(X_val)
print(metrics.f1_score(y_val.values, y_pred)) 
print(classification_report(y_val.values,y_pred))
y_pred_train_2 = xgbr.predict(X_train_2)

0.2978723404255319
             precision    recall  f1-score   support

          0       0.86      1.00      0.92       201
          1       0.88      0.18      0.30        39

avg / total       0.86      0.86      0.82       240



D:\Users\DELL\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\Users\DELL\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [20]:
# 模型调参 xgboost
from xgboost import XGBClassifier
xgbr = XGBClassifier(scale_pos_weight=6)
xgbr.fit(X_train, y_train)
y_pred = xgbr.predict(X_val)
print(metrics.f1_score(y_val.values, y_pred)) 
print(classification_report(y_val.values,y_pred))
y_pred_train_2 = xgbr.predict(X_train_2)

0.36666666666666664
             precision    recall  f1-score   support

          0       0.87      0.95      0.91       201
          1       0.52      0.28      0.37        39

avg / total       0.82      0.84      0.82       240



D:\Users\DELL\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
D:\Users\DELL\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


#### 3 、预测

In [17]:
X_train_2['Churn'] = y_pred_train_2
X_train['Churn'] = y_train

data_train = pd.concat([X_train,X_train_2],axis=0)
data_train['Churn'] .value_counts()

0    3293
1     467
Name: Churn, dtype: int64

In [19]:
from sklearn.model_selection import train_test_split
X = data_train.iloc[:, :-1]
y = data_train.iloc[:,-1]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=8)

In [20]:
def prod_test_data(path):

    """
    处理测试集
    :return: 
    """
    df = pd.read_csv(path)
    
    X_test_df_index = df['id']
    df =df.loc[:,[ 'account_length', 'international_plan', 'voice_mail_plan', 'number_vmail_messages',
           'total_day_minutes', 'total_day_calls', 'total_day_charge',
           'total_eve_minutes', 'total_eve_calls', 'total_eve_charge',
           'total_night_minutes', 'total_night_calls', 'total_night_charge',
           'total_intl_minutes', 'total_intl_calls', 'total_intl_charge']]
    df['international_plan'] = df['international_plan'].map(lambda x:1 if x==' yes' else 0)
    df['voice_mail_plan'] = df['voice_mail_plan'].map(lambda x:1 if x==' yes' else 0)
    
    data_test = df    
    X_test = data_test.iloc[:,:]

    
    return X_test,X_test_df_index

In [21]:
X_test,X_test_df_index= prod_test_data(path_test)

In [59]:
# 模型
from xgboost import XGBClassifier
# class_weight = dict({0:14, 1:1})
# sample_weight = pd.DataFrame(y_train)['SeriousDlqin2yrs'].map(lambda x:0.99 if x==1 else 0.01)
# alpha
#     L1正则化系数，默认为1
# lambda


xgbr = XGBClassifier(scale_pos_weight=2)
# xgbr = XGBClassifier()
xgbr.fit(X_train, y_train)
y_pred = xgbr.predict(X_val)
print(metrics.f1_score(y_val.values, y_pred)) 
print(classification_report(y_val.values, y_pred))

0.7373737373737372
             precision    recall  f1-score   support

          0       0.96      0.98      0.97       834
          1       0.79      0.69      0.74       106

avg / total       0.94      0.94      0.94       940



D:\Users\DELL\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [26]:
# from catboost import CatBoostClassifier
# cgbr = CatBoostClassifier(iterations=250,depth=5,learning_rate=0.01,l2_leaf_reg=1)
# cgbr.fit(X_train, y_train)

In [56]:
# 模型预测结果融合
Id_pred=pd.DataFrame()
Id_pred['ID'] = X_test_df_index

Id_pred['pred_xgbr']=xgbr.predict(X_test)
# Id_pred['pred_lgbr']=lgbr.predict(X_test)
# Id_pred['pred_cgbr']=cgbr.predict(X_test)

Id_pred['Predicted_Results'] = Id_pred['pred_xgbr']
Id_pred['Predicted_Results'] = Id_pred['Predicted_Results'].map(lambda x: 'True' if x==1 else 'False' )
del Id_pred['pred_xgbr']

Id_pred.to_csv("./submit/XXX_results.csv",index=None)

D:\Users\DELL\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [57]:
Id_pred.Predicted_Results.value_counts()

False    909
True      91
Name: Predicted_Results, dtype: int64